In [1]:
import os
import pandas as pd
import polars as pl
import numpy as np
import tensorflow as tf


env = "local"

In [2]:
if env == "local":
    os.chdir("/Users/samuel/Documents/GitHub/QB-GPT/")
else:
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir("/content/gdrive/MyDrive/NFL_Challenge/NFL-GPT/NFL data")

In [3]:
os.listdir()

['data_models',
 '.DS_Store',
 'LICENSE',
 'models',
 'README.md',
 '.gitignore',
 '.gitattributes',
 'data_preprocessing',
 'index',
 '.git',
 'notebooks']

In [4]:
testing_data = tf.data.Dataset.load("data_models/QBGPT/test_tokens_NFL_GPT")

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 8351411500713370602


In [5]:
testing_data

<_LoadDataset element_spec=({'OffDef': TensorSpec(shape=(256,), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.int64, name=None), 'position_ids': TensorSpec(shape=(256,), dtype=tf.int64, name=None), 'scrim_ids': TensorSpec(shape=(256,), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(256,), dtype=tf.int64, name=None), 'start_ids': TensorSpec(shape=(256,), dtype=tf.int64, name=None), 'input_ids': TensorSpec(shape=(256,), dtype=tf.int64, name=None), 'PlayType': TensorSpec(shape=(256,), dtype=tf.int64, name=None), 'pos_ids': TensorSpec(shape=(256,), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(256,), dtype=tf.int64, name=None), 'side_ids': TensorSpec(shape=(256,), dtype=tf.int64, name=None)}, TensorSpec(shape=(256,), dtype=tf.int64, name=None))>

In [6]:
moves_index = pl.read_parquet("index/moves_index.parquet")
position_index = pl.read_parquet("index/positions_index.parquet")
starts_index = pl.read_parquet("index/starts_index.parquet")
scrim_index = pl.read_parquet("index/scrimmage_index.parquet")
playtype_index = pl.read_parquet("index/plays_index.parquet")

In [7]:
most_prob_pos = pl.read_parquet("models/generating/most_probable_pos.parquet")
most_prob_start = pl.read_parquet("models/generating/most_probable_starts.parquet")

In [8]:
position_index

position,position_ID,Cat
str,i64,str
"""C""",0,"""Pos"""
"""CB""",1,"""Pos"""
"""DB""",2,"""Pos"""
"""DE""",3,"""Pos"""
"""DL""",4,"""Pos"""
"""DT""",5,"""Pos"""
"""FB""",6,"""Pos"""
"""FS""",7,"""Pos"""
"""G""",8,"""Pos"""


In [9]:
starts_index

Starting_x,Starting_y,Start_ID,Cat
i64,i64,i32,str
-20,-22,0,"""Start"""
-20,-21,1,"""Start"""
-20,-20,2,"""Start"""
-20,-19,3,"""Start"""
-20,-18,4,"""Start"""
-20,-17,5,"""Start"""
-20,-16,6,"""Start"""
-20,-15,7,"""Start"""
-20,-14,8,"""Start"""


In [10]:
starts_index.filter(pl.col("Start_ID") == 1981)

Starting_x,Starting_y,Start_ID,Cat
i64,i64,i32,str


In [11]:
most_prob_pos.filter(pl.col("OffDef_ID") == 1)

OffDef_ID,position_ID,count
i32,i64,u32
1,27,90901
1,26,57220
1,21,42631
1,25,28958
1,20,27715
1,8,25728
1,12,16494
1,17,13668
1,0,12794


In [12]:
most_prob_start.filter(pl.col("position_ID") == 4)

position_ID,Start_ID,count
i64,i32,u32
4,1004,54
4,1031,45
4,1006,44
4,1005,35
4,1003,24
4,1001,22
4,950,21
4,1009,19


In [13]:
starting_point = {
    "OffDef" : [1 for i in range(11)],
    "attention_mask" : [1 for i in range(11)],
    "position_ids" : [27, 27, 20, 0, 1, 2, 7, 13, 8, 20, 20],
    "scrim_ids" : [0 for i in range(11)],
    "start_ids" : [994, 993, 997, 996, 990, 994, 988, 990, 997, 998, 991],
    "input_ids" : [11164 for i in range(11)],
    "PlayType" : [4 for i in range(11)],
    "pos_ids" : [0 for i in range(11)],
    "token_type_ids" : [1 for i in range(11)]
}

starting_point = {k: np.expand_dims(np.array(v), axis = 0) for k,v in starting_point.items()}

In [14]:
playtype_index

PlayType,PlayType_ID
str,i64
"""extra_point""",0
"""field_goal""",1
"""kickoff""",2
"""no_play""",3
"""pass""",4
"""punt""",5
"""qb_spike""",6
"""run""",7


In [15]:
starting_point

{'OffDef': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'position_ids': array([[27, 27, 20,  0,  1,  2,  7, 13,  8, 20, 20]]),
 'scrim_ids': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'start_ids': array([[994, 993, 997, 996, 990, 994, 988, 990, 997, 998, 991]]),
 'input_ids': array([[11164, 11164, 11164, 11164, 11164, 11164, 11164, 11164, 11164,
         11164, 11164]]),
 'PlayType': array([[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]]),
 'pos_ids': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'token_type_ids': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [16]:
from models.modeling.QBGPT.models import QBGPT, AttentionBlock

moves_to_pred = 11164
input_size = 11166
starts_size = 1985
scrimmage_size = 100
positions_id = 29
temp_ids = 52


model_tiny = QBGPT(input_vocab_size = input_size,
                    positional_vocab_size = temp_ids,
                    position_vocab_size=positions_id,
                    start_vocab_size=starts_size,
                    scrimmage_vocab_size=scrimmage_size,
                    offdef_vocab_size = 2,
                    type_vocab_size = 3,
                    playtype_vocab_size = 9,
                    embedding_dim = 64,
                    hidden_dim = 64,
                    to_pred_size = moves_to_pred)



model_tiny.load_weights("models/modeling/QBGPT/weights/model_tiny/QBGPT")

In [19]:
tf.argmax(model_tiny(starting_point), axis = -1)

<tf.Tensor: shape=(1, 11), dtype=int64, numpy=array([[5273, 5273, 5522, 5522, 5273, 5273, 5273, 5522, 5522, 5522, 5522]])>

In [15]:
essai = list(testing_data)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 8351411500713370602


In [16]:
essai_spe = essai[0][0]

In [17]:
trial = {k : np.expand_dims(np.array([v[i] for i in range(len(v)) if essai_spe["pos_ids"][i] < 48]), axis = 0) for k,v in essai_spe.items()}

In [18]:
trial["pos_ids"]

array([[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 25,
        26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 25, 26,
        27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 25, 26, 27,
        28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 25, 26, 27, 28,
        29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 25, 26, 27, 28, 29,
        30, 31, 32, 33, 34, 35, 36, 37, 38, 39]])

In [19]:
trial["position_ids"]

array([[26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 20, 20, 20, 20, 20,
        20, 20, 20, 20, 20, 20, 20, 20, 20, 20]])

In [20]:
trial["input_ids"]

array([[4348, 4349, 4350, 4351, 4352, 4353, 4435, 4436, 4437, 4437, 4438,
        4438, 4439, 4439, 4439, 3299, 3123, 3034, 2945, 2765, 2676, 2584,
        2492, 2401, 2401, 2311, 2219, 2125, 2125, 2030, 4516, 4517, 4519,
        4520, 4435, 4437, 4356, 4357, 4273, 4190, 4109, 4026, 3942, 3855,
        3765, 5114, 5116, 5117, 5118, 5120, 5121, 5122, 5123, 5039, 5039,
        5040, 4955, 4956, 4873, 4873, 5505, 5505, 5423, 5423, 5423, 5340,
        5340, 5258, 5258, 5259, 5171, 5172, 5172, 5085, 5085, 5764, 5764,
        5764, 5764, 5764, 5764, 5682, 5682, 5682, 5599, 5599, 5518, 5519,
        5519, 5437]])

In [21]:
trial["labels"]

array([[4349, 4350, 4351, 4352, 4353, 4435, 4436, 4437, 4437, 4438, 4438,
        4439, 4439, 4439, -100, 3123, 3034, 2945, 2765, 2676, 2584, 2492,
        2401, 2401, 2311, 2219, 2125, 2125, 2030, -100, 4517, 4519, 4520,
        4435, 4437, 4356, 4357, 4273, 4190, 4109, 4026, 3942, 3855, 3765,
        -100, 5116, 5117, 5118, 5120, 5121, 5122, 5123, 5039, 5039, 5040,
        4955, 4956, 4873, 4873, -100, 5505, 5423, 5423, 5423, 5340, 5340,
        5258, 5258, 5259, 5171, 5172, 5172, 5085, 5085, -100, 5764, 5764,
        5764, 5764, 5764, 5682, 5682, 5682, 5599, 5599, 5518, 5519, 5519,
        5437, -100]])

In [22]:
tf.argmax(model_tiny(trial), axis = -1)

<tf.Tensor: shape=(1, 90), dtype=int64, numpy=
array([[4348, 4264, 4265, 3663, 4266, 3839, 3749, 3839, 3838, 3838, 3838,
        3838, 4439, 5263, 4439, 3122, 2943, 5841, 3660, 2675, 6107, 6698,
        6781, 6782, 8823, 6781, 2125, 4005, 4005, 6365, 4516, 4517, 4087,
        4521, 3749, 4003, 3838, 3838, 6106, 4190, 4025, 3308, 3942, 3572,
        7477, 5517, 5117, 5090, 4948, 5035, 5675, 6611, 6531, 1425, 1425,
        3749, 4673, 3663, 3663, 4874, 6188, 6188, 5525, 6104, 6104, 6273,
        5258, 6271, 5170, 3395, 6272, 3748, 3748, 5680, 5085, 5763, 5763,
        5764, 5764, 5764, 5764, 5682, 5682, 5682, 5599, 5599, 5520, 5520,
        5520, 5354]])>

In [24]:
moves_index.filter(pl.col("Zone_ID").is_in([4350, 4264]))

x,y,Zone_ID,Cat
i64,i64,i64,str
-15,7,4264,"""Moves"""
-14,8,4350,"""Moves"""
